# Modeling Fasting Probability with Bayesian Logistic Regression
This notebook models the probability of being in a fasting state using Bayesian logistic regression with PyMC and Metropolis-Hastings sampling.

## ⚠️ Data Notice
*The original dataset `freezer.txt` used in this project is not available. However, the code is preserved below and ready to run with any compatible dataset.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import os

## Logistic Function Definition

In [ ]:
def logistic(x, beta, alpha):
    return 1 / (1 + np.exp(beta * x + alpha))

## Load Dataset
Assumes a file `freezer.txt` with two columns: time and binary variable indicating fasting (1) or not (0).

In [ ]:
# ⚠️ Data loading is commented out due to missing file
# current_dir = os.path.dirname(os.path.abspath("freezer.txt"))
# file_path = os.path.join(current_dir, 'freezer.txt')
# data = np.loadtxt(file_path)
# x = data[:, 0]
# y = data[:, 1]

## Bayesian Logistic Regression Model

In [ ]:
if 'x' in locals() and 'y' in locals():
    with pm.Model() as eat_model:
        alpha = pm.Normal('alpha', mu=0.0, sigma=1.0, initval=0.0)
        beta = pm.Normal('beta', mu=0.0, sigma=1.0, initval=0.0)

        p = pm.Deterministic('p', 1. / (1. + pm.math.exp(beta * x + alpha)))
        observed = pm.Bernoulli('obs', p, observed=y)

        step = pm.Metropolis()
        eat_trace = pm.sample(2000, step=step, tune=1000)

## Posterior Analysis

In [ ]:
if 'eat_trace' in locals():
    plt.rcParams.update({'axes.labelsize': 8, 'xtick.labelsize': 8, 'ytick.labelsize': 8, 'font.size': 8})
    pm.plot_trace(eat_trace, var_names=['alpha', 'beta'])
    plt.show()
else:
    print("⚠️ 'eat_trace' not defined. Provide data and run the model cell first.")

## Probability Estimation and Plotting

In [ ]:
if 'eat_trace' in locals():
    alpha_samples = eat_trace.posterior['alpha'].values
    beta_samples = eat_trace.posterior['beta'].values
    alpha_mean = alpha_samples.mean()
    beta_mean = beta_samples.mean()

    time_est = np.linspace(x.min() - 5, x.max() + 5, 1000)
    eat_est = logistic(time_est, beta_mean, alpha_mean)

    plt.figure(figsize=(12, 6))
    plt.plot(time_est, eat_est, color='orange', linewidth=6)
    plt.xlabel('Tiempo (minutos)', fontsize=24)
    plt.ylabel('Probabilidad', fontsize=24)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.legend()
    plt.grid(False)
    plt.show()
else:
    print("⚠️ Cannot estimate probabilities. Trace not available.")

## Summary
This notebook demonstrates Bayesian logistic regression for binary classification of fasting state. It uses MCMC with Metropolis sampling. While the original dataset is not available, the code structure remains intact and reusable.